In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [18]:
# basic data processing and cleaning
# note: we manually changed the header line of the csv,
# removing the spaces between commas and modifying the last column name

trials = pd.read_csv("../data/processed_data.csv")
#print(df[:10])
print(list(trials.columns.values))
#print(np.size(trials)) # 327250

# get rid of bugged data, which amounts to 9416 records
trials = trials[trials['Bugged'] != "Bugged"] 


['MTurk ID', 'Sentence Number', 'List Number', 'Subset Number', 'Trial Number', 'Target Sentence', 'Scrambled Sentence', 'Produced Sentence', 'Time (ms)', 'Time', 'Bugged']


In [25]:
# add column for whether produced matches target
def ProdMatchesTgt(dataframe_row):
    if dataframe_row['Target Sentence'] == dataframe_row['Produced Sentence']:
        return True
    else:
        return False
        
trials['ProdMatchesOrig'] = trials.apply(ProdMatchesTgt, axis=1)    
#print(trials[:10])


In [ ]:
# add column for longest matching substring
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Longest_common_substring#Python_3
# https://en.wikipedia.org/wiki/Longest_common_substring_problem

#def longestMatchingSubstring( dataframe_row ):
    
 

In [39]:
# proportion that match original
# For each of 30,000 trials, identify the proportion of word pairs that match the original sentence 
# in order (we'll have to figure out what to do with duplicates, which are common. esp. for function 
# words; perhaps we should restrict things to content words for starters).

# I'm going to interpret this as: how many bigrams from the target appeared in the production
# divided by the total number of bigrams in the target

# bypassing efficiency for now
def getBigrams( string ):
    bigs = []
    sep = string.lower().split()
    for k,v in enumerate(sep):
        if k == len(sep) - 1:
            break
        else:
            bigs.append((v,sep[k+1]))
    return bigs

def searchForBigrams( bigrams_prod, bigrams_tgt ):
    ''' bigrams is a list of tuples of word pairs, returns number of bigrams from tgt in prod
        divided by total number in prod 
    '''
    # set intersection
    bigList = [bigrams_prod,bigrams_tgt]
    bigList = [set(a) for a in bigList]
    
    numProdBig = len(bigList[1])
    intersectionTgtBig = set.intersection(*bigList)

    return len( intersectionTgtBig ) / numProdBig

# test
#b1 = getBigrams("this is a sample sentence")
#b2 = getBigrams("this is a very cool sample sentence")
#print( searchForBigrams( b1,b2))  # works out all right!
    
def bigramDriver(dataframe_row):
    b1 = getBigrams(dataframe_row['Produced Sentence'])
    b2 = getBigrams(dataframe_row['Target Sentence'])
    
    proportion = searchForBigrams(b1,b2)
    return proportion

trials['proportionMatchingBigram'] = trials.apply(bigramDriver, axis=1)

print(trials[:5])

         MTurk ID  Sentence Number  List Number  Subset Number  Trial Number  \
1  A30RAYNDOWQ61S               56            3              2             2   
2  A30RAYNDOWQ61S               51            3              2             3   
3  A30RAYNDOWQ61S               91            3              2             4   
4  A30RAYNDOWQ61S               58            3              2             5   
5  A30RAYNDOWQ61S               83            3              2             6   

                                     Target Sentence  \
1  the blast shattered the windows of the villa a...   
2  the equipment was often moved quickly when ins...   
3  evidence from abroad showed students were prep...   
4  south africa has openly expressed willingness ...   
5  the regular bus service had been suspended bec...   

                                  Scrambled Sentence  \
1  neighbouring windows shattered the other the h...   
2  the to quickly equipment inspectors moved were...   
3  showed stud

In [42]:
# edit distance between production and original
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python

# this might actually need to be optimized
def levenshtein(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
                
        return v1[len(t)]

    
def editDriver( dataframe_row ):
    a = dataframe_row['Produced Sentence']
    b = dataframe_row['Target Sentence']
    
    return levenshtein( a, b )

s = "the equipment was often moved quickly when inspectors were about to arrive"
t = "equipment was often quickly moved when the inspectors were about to arrive"
print(levenshtein(s,t))

#trials['editDist'] = trials.apply(editDriver, axis=1)

#print(trials[:5])

20
